# 3. Feature Engineering and Modeling



## 3.1 Imports

In [1]:
#!pip install category_encoders dirty_cat feature-engine

In [2]:
import datetime
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, scale, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.inspection import PartialDependenceDisplay
from scipy.stats import uniform, randint
import category_encoders
import dirty_cat
import feature_engine.datetime
from IPython.display import Markdown as md

## 3.2 Load the Data

In [3]:
dog_df = pd.read_csv('../clean_data/austin_dogs_eda_clean.csv', index_col=0)

In [4]:
dog_df.shape

(55445, 10)

In [5]:
dog_df.head()

,name,breed_category,sex_intake,intake_type,condition_intake,outcome,delta_time_d,month_outcome,year_outcome,age_in_years_bins
0,*Brock,Beagle,Neutered Male,Stray,Normal,Transfer,4.952778,1,2019,2-3 yrs
1,Belle,Spaniel,Spayed Female,Stray,Normal,Return to Owner,0.093056,7,2015,8-9 yrs
2,Runster,Basenji,Intact Male,Stray,Normal,Return to Owner,6.940278,4,2016,0-1 yrs
3,Rio,Doberman,Neutered Male,Stray,Normal,Return to Owner,3.151389,7,2014,4-5 yrs
4,Odin,Labrador Retriever,Neutered Male,Owner Surrender,Normal,Return to Owner,3.206944,2,2017,2-3 yrs


In [6]:
dog_df.describe()

,delta_time_d,month_outcome,year_outcome
count,55445.000000,55445.000000,55445.000000
mean,17.135873,6.531175,2017.117955
std,46.483790,3.521655,2.405349
min,0.000694,1.000000,2013.000000
25%,2.856250,3.000000,2015.000000
50%,5.215972,7.000000,2017.000000
75%,11.898611,10.000000,2019.000000
max,1912.938194,12.000000,2022.000000


In [7]:
dog_df.describe(include=['O'])

,name,breed_category,sex_intake,intake_type,condition_intake,outcome,age_in_years_bins
count,43726,55445,55445,55445,55445,55445,55445
unique,14084,52,4,6,13,8,11
top,Bella,Labrador Retriever,Intact Male,Stray,Normal,Adoption,0-1 yrs
freq,326,8986,21318,41032,49186,25963,28497


In [8]:
dog_df['name'].isnull().sum()

11719

In [9]:
dog_df['name'].fillna('Unknown', inplace=True)

## 3.3 Feature Engineering and Modeling

#### 3.3.1 Creating a Naive Baseline

We begin by creating a Naive Baseline to compare against future models. The data is split into X and y variables; X being all the features except our predicted values, and y being only the values for delta_time_d, the time spent in the shelter in days. We randomly split the data into test and train variables and calculate the mean absolute error of each y_test value against the y_train mean.

In [10]:
X = dog_df.drop('delta_time_d', axis=1)
y = dog_df.delta_time_d

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [12]:
print(f"The average time spent in the shelter in the training set is \033[1m{y_train.mean() :.3f}\033[0m days.")

The average time spent in the shelter in the training set is 17.050 days.


In [13]:
y_guesses = [y_train.mean()] * len(y_test)
error = mean_absolute_error(y_test, y_guesses)
print(f'On average, each of these "predictions" was off by \033[1m{error :,.3f}\033[0m days.')

On average, each of these "predictions" was off by 19.692 days.


In [14]:
md(f'''On average, predicting each y_test value against its mean gives us a mean absolute error of around **{error:.3f} days**.

This is our Naive baseline and we begin to create and tune future models to predict with a mean absolute error of less.''')

On average, predicting each y_test value against its mean gives us a mean absolute error of around **19.692 days**.

This is our Naive baseline and we begin to create and tune future models to predict with a mean absolute error of less.

#### 3.3.2 Simple Linear Regression Model

Next we make a simple Linear Regression model to see if there are any linear trends in the data. We remove the high cardinality features from our set when we use OneHotEncoder to simplify this linear model even more and prevent overfitting.

In [15]:
basic_features = ['month_outcome', 'year_outcome', 'condition_intake', 'outcome', 'intake_type', 'sex_intake']

simple_linear_model = make_pipeline(
    OneHotEncoder(handle_unknown='ignore'),
    SimpleImputer(strategy='mean'),
    LinearRegression()
)
simple_linear_model.fit(X_train[basic_features], y_train)

y_pred = simple_linear_model.predict(X_test[basic_features])

error = mean_absolute_error(y_test, y_pred)

print(f'On average, each of the test set predictions was off by \033[1m{error :,.3f}\033[0m days. This is the second baseline to beat.')
print(f'Any more complicated model should have error less than \033[1m{error :,.3f}\033[0m to be better than the simple model.')

On average, each of the test set predictions was off by 18.562 days. This is the second baseline to beat.
Any more complicated model should have error less than 18.562 to be better than the simple model.


Our basic linear model predicts a mean absolute error that is slightly better than our naive baseline with an average prediction of 1 day more accurate. We know that based on our exploratory data analysis that our features did not have linear correlations, however, this simple linear model is still a better predictor than our naive baseline.

#### 3.3.3 Random Forest Regressor Model

A random forest regressor model is supervised learning model that uses ensemble methods, or multiple machine learning algorithms to make a more accurate prediction than a single model.

A random forest acts as an estimator algorithm that aggregates the result of many decision trees and then outputs the most optimal result. We run our features (except name) through the OrdinalEncoder, which handles high cardinal categorical data better than OneHotEncoding.

In [16]:
features = ['month_outcome', 'year_outcome', 'condition_intake', 'outcome', 'intake_type', 
                  'sex_intake', 'breed_category', 'age_in_years_bins']

simple_rfr = make_pipeline(
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=100000000),
    SimpleImputer(strategy='median'),
    RandomForestRegressor()
)

simple_rfr.fit(X_train[features], y_train)
y_pred = simple_rfr.predict(X_test[features])

error = mean_absolute_error(y_test, y_pred)
error2 = mean_absolute_percentage_error(y_test, y_pred)

print(f'With our simple Random Forest Regressor model, on average, each of the test set predictions was off by \n\033[1m{error :,.3f} or {error2:,.3f}%.')

With our simple Random Forest Regressor model, on average, each of the test set predictions was off by 
17.016 or 10.901%.


Compared to our simple Linear Regression model, our test set predictions are have a mean absolute error of around 16, which is about 2 less than our previous model prediction.

#### 3.3.4 Random Forest Regressor Model using the dirty_cat library

We can increase the efficiency of our pre-processing using the dirty_cat (or dirty categories) library. This library has ways to transform our data for processing high cardinality string categorical features.

In [17]:
random_forest_model = make_pipeline(
    dirty_cat.SuperVectorizer(
    cardinality_threshold=50,
    low_card_cat_transformer=category_encoders.CountEncoder(handle_unknown=0),
    high_card_cat_transformer=dirty_cat.GapEncoder(n_components=10),
    numerical_transformer=SimpleImputer(strategy='median'),
    datetime_transformer=feature_engine.datetime.DatetimeFeatures(missing_values='ignore')),
    StandardScaler(),
    RandomForestRegressor()
)

In [18]:
random_forest_model.fit(X_train, y_train)

Pipeline(steps=[('supervectorizer',
                 SuperVectorizer(cardinality_threshold=50,
                                 datetime_transformer=DatetimeFeatures(missing_values='ignore'),
                                 high_card_cat_transformer=GapEncoder(),
                                 low_card_cat_transformer=CountEncoder(combine_min_nan_groups=True,
                                                                       handle_unknown=0),
                                 numerical_transformer=SimpleImputer(strategy='median'))),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

In [19]:
y_pred = random_forest_model.predict(X_test)

# Compare the difference between the true salaries versus the predictions
error = mean_absolute_error(y_test, y_pred)
error2 = mean_absolute_percentage_error(y_test, y_pred)

print(f'After using dirty_cat transformations on our random forest regressor model, on average, \n\
each of the test set predictions was off by \033[1m{error :,.3f} or {error2:,.3f}%.')

After using dirty_cat transformations on our random forest regressor model, on average, 
each of the test set predictions was off by 16.478 or 10.510%.


After using dirty_cat on our data, we can decrease our mean absolute error by another 0.5 days on average.

#### 3.3.5 HistGradientBoostingRegressor

HistGradientBoostingRegressor is another ensemble method machine learning algorithm that also uses regression trees. However, this model also has ways to actively learn from missing NaN values that can contribute to growth. We try this model to see if it can learn from all the missing NaN values from the 'name' feature.

In [20]:
hgbr_model = make_pipeline(
    dirty_cat.SuperVectorizer(
        cardinality_threshold=50,
        low_card_cat_transformer=category_encoders.CountEncoder(handle_unknown=0),
        high_card_cat_transformer=dirty_cat.GapEncoder(n_components=10),
        numerical_transformer=StandardScaler(),
        datetime_transformer=feature_engine.datetime.DatetimeFeatures(missing_values='ignore')
    ),
    HistGradientBoostingRegressor(
        max_iter=100,
        early_stopping=False,
        scoring='mean_absolute_error',
        validation_fraction=0.1,
        n_iter_no_change=10,
        verbose=1,
        random_state=0
    )
)

hgbr_model.fit(X_train, y_train)

Binning 0.009 GB of training data: 0.083 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 31 leaves, max depth = 12, in 0.011s
[2/100] 1 tree, 31 leaves, max depth = 10, in 0.012s
[3/100] 1 tree, 31 leaves, max depth = 10, in 0.013s
[4/100] 1 tree, 31 leaves, max depth = 12, in 0.013s
[5/100] 1 tree, 31 leaves, max depth = 9, in 0.012s
[6/100] 1 tree, 31 leaves, max depth = 11, in 0.010s
[7/100] 1 tree, 31 leaves, max depth = 11, in 0.010s
[8/100] 1 tree, 31 leaves, max depth = 12, in 0.010s
[9/100] 1 tree, 31 leaves, max depth = 10, in 0.010s
[10/100] 1 tree, 31 leaves, max depth = 10, in 0.011s
[11/100] 1 tree, 31 leaves, max depth = 10, in 0.010s
[12/100] 1 tree, 31 leaves, max depth = 11, in 0.009s
[13/100] 1 tree, 31 leaves, max depth = 12, in 0.011s
[14/100] 1 tree, 31 leaves, max depth = 10, in 0.009s
[15/100] 1 tree, 31 leaves, max depth = 12, in 0.010s
[16/100] 1 tree, 31 leaves, max depth = 12, in 0.010s
[17/100] 1 tree, 31 leaves, max depth = 12, in 0.010s
[18/100] 1 tree,

Pipeline(steps=[('supervectorizer',
                 SuperVectorizer(cardinality_threshold=50,
                                 datetime_transformer=DatetimeFeatures(missing_values='ignore'),
                                 high_card_cat_transformer=GapEncoder(),
                                 low_card_cat_transformer=CountEncoder(combine_min_nan_groups=True,
                                                                       handle_unknown=0),
                                 numerical_transformer=StandardScaler())),
                ('histgradientboostingregressor',
                 HistGradientBoostingRegressor(early_stopping=False,
                                               random_state=0,
                                               scoring='mean_absolute_error',
                                               verbose=1))])

In [21]:
y_pred = hgbr_model.predict(X_test)

# Compare the difference between the true salaries versus the predictions
error = mean_absolute_error(y_test, y_pred)
error2 = mean_absolute_percentage_error(y_test, y_pred)

print(f'After using dirty_cat transformations on our HistGradientBoostingRegressor model, on average, \n\
each of the test set predictions was off by \033[1m{error :,.3f} or {error2 :,.3f}%.')

After using dirty_cat transformations on our HistGradientBoostingRegressor model, on average, 
each of the test set predictions was off by 15.859 or 13.316%.


In [22]:
y_tr_pred = hgbr_model.predict(X_train)
y_te_pred = hgbr_model.predict(X_test)

In [23]:
median_r2 = r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)
median_r2

(0.42085704289831793, 0.17881972366863785)

In [24]:
median_mae = mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)
median_mae

(13.969329620480389, 15.85863679170765)

In [25]:
median_mape = mean_absolute_percentage_error(y_train, y_tr_pred), mean_absolute_percentage_error(y_test, y_te_pred)
median_mape

(11.611441890730859, 13.319080231147764)

We see an average 11.7% difference in the forecasted value and the actual value in our training set.

We also see an average 11.08% difference in the forecasted value and the actual value in our test set.



#### 3.3.6 Cross Validation

In [26]:
scoring = ['neg_mean_absolute_error', 'neg_mean_absolute_percentage_error']
cv_results = cross_validate(hgbr_model, X_train, y_train, cv=5, scoring=scoring)

Binning 0.007 GB of training data: 0.088 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 31 leaves, max depth = 10, in 0.014s
[2/100] 1 tree, 31 leaves, max depth = 10, in 0.014s
[3/100] 1 tree, 31 leaves, max depth = 12, in 0.014s
[4/100] 1 tree, 31 leaves, max depth = 11, in 0.013s
[5/100] 1 tree, 31 leaves, max depth = 10, in 0.012s
[6/100] 1 tree, 31 leaves, max depth = 11, in 0.013s
[7/100] 1 tree, 31 leaves, max depth = 12, in 0.013s
[8/100] 1 tree, 31 leaves, max depth = 11, in 0.014s
[9/100] 1 tree, 31 leaves, max depth = 11, in 0.013s
[10/100] 1 tree, 31 leaves, max depth = 12, in 0.010s
[11/100] 1 tree, 31 leaves, max depth = 14, in 0.014s
[12/100] 1 tree, 31 leaves, max depth = 9, in 0.013s
[13/100] 1 tree, 31 leaves, max depth = 12, in 0.014s
[14/100] 1 tree, 31 leaves, max depth = 10, in 0.017s
[15/100] 1 tree, 31 leaves, max depth = 11, in 0.013s
[16/100] 1 tree, 31 leaves, max depth = 12, in 0.014s
[17/100] 1 tree, 31 leaves, max depth = 10, in 0.021s
[18/100] 1 tree,

[47/100] 1 tree, 31 leaves, max depth = 13, in 0.011s
[48/100] 1 tree, 31 leaves, max depth = 11, in 0.010s
[49/100] 1 tree, 31 leaves, max depth = 10, in 0.010s
[50/100] 1 tree, 31 leaves, max depth = 14, in 0.009s
[51/100] 1 tree, 31 leaves, max depth = 10, in 0.009s
[52/100] 1 tree, 31 leaves, max depth = 12, in 0.009s
[53/100] 1 tree, 31 leaves, max depth = 8, in 0.010s
[54/100] 1 tree, 31 leaves, max depth = 14, in 0.009s
[55/100] 1 tree, 31 leaves, max depth = 11, in 0.010s
[56/100] 1 tree, 31 leaves, max depth = 12, in 0.009s
[57/100] 1 tree, 31 leaves, max depth = 11, in 0.009s
[58/100] 1 tree, 31 leaves, max depth = 11, in 0.010s
[59/100] 1 tree, 31 leaves, max depth = 12, in 0.010s
[60/100] 1 tree, 31 leaves, max depth = 13, in 0.009s
[61/100] 1 tree, 31 leaves, max depth = 10, in 0.011s
[62/100] 1 tree, 31 leaves, max depth = 9, in 0.009s
[63/100] 1 tree, 31 leaves, max depth = 12, in 0.009s
[64/100] 1 tree, 31 leaves, max depth = 11, in 0.009s
[65/100] 1 tree, 31 leaves, ma

[36/100] 1 tree, 31 leaves, max depth = 18, in 0.009s
[37/100] 1 tree, 31 leaves, max depth = 14, in 0.009s
[38/100] 1 tree, 31 leaves, max depth = 10, in 0.009s
[39/100] 1 tree, 31 leaves, max depth = 15, in 0.009s
[40/100] 1 tree, 31 leaves, max depth = 11, in 0.009s
[41/100] 1 tree, 31 leaves, max depth = 15, in 0.010s
[42/100] 1 tree, 31 leaves, max depth = 14, in 0.009s
[43/100] 1 tree, 31 leaves, max depth = 9, in 0.009s
[44/100] 1 tree, 31 leaves, max depth = 11, in 0.009s
[45/100] 1 tree, 31 leaves, max depth = 10, in 0.010s
[46/100] 1 tree, 31 leaves, max depth = 10, in 0.009s
[47/100] 1 tree, 31 leaves, max depth = 9, in 0.009s
[48/100] 1 tree, 31 leaves, max depth = 11, in 0.009s
[49/100] 1 tree, 31 leaves, max depth = 14, in 0.011s
[50/100] 1 tree, 31 leaves, max depth = 7, in 0.011s
[51/100] 1 tree, 31 leaves, max depth = 11, in 0.010s
[52/100] 1 tree, 31 leaves, max depth = 12, in 0.010s
[53/100] 1 tree, 31 leaves, max depth = 11, in 0.010s
[54/100] 1 tree, 31 leaves, max

In [27]:
cv_scores = cv_results.get('test_neg_mean_absolute_error')
cv_scores

array([-15.74763031, -15.98827723, -15.08380825, -15.89085077,
       -15.40095038])

In [28]:
np.abs(np.mean(cv_scores)), np.std(cv_scores)

(15.622303386255567, 0.33488372863112253)

## 3.4 Summary



In conclusion, the HistGradientBoostingRegressor algorithm produced the best mean absolute error. We chose this metric to evaluate each model's performance due to its resilience against outliers in the data set. Since we have many outliers that are over 1.5 IQR away from our 75th percentile, evaluating the model one this metric will be more accurate than using r2 or mean squared error.

Our best model is the HistGradientBoosterRegressor model that produces predictions with the lowest Mean Absolute Error compared to our baseline and the rest of our tested models.

From this model, we can use it to predict the estimated time spent in the shelter for dogs based on features such as their name, age, breed, intake condition, the time of year, etc. This model can be further used with unknown data to predict the estimated times to help with the logistics incoming and outgoing dogs in a shelter with limited and most often full capacity.


In [29]:
patches = dog_df.iloc[22279]
patches

name                       Patches
breed_category            Pit Bull
sex_intake           Spayed Female
intake_type                  Stray
condition_intake           Injured
outcome                   Adoption
delta_time_d           1912.938194
month_outcome                    4
year_outcome                  2021
age_in_years_bins        10-11 yrs
Name: 22279, dtype: object

In [30]:
patches_x = np.array(patches.drop('delta_time_d'))
patches_x = np.expand_dims(patches_x, 0)
patches_y = patches.delta_time_d


In [31]:
patches_y_pred = hgbr_model.predict(patches_x)
patches_y_pred

array([687.38678592])

Looking at Patches again, our model determined that the time spent in the shelter for Patches would be 701 days based off her features. The reality of her situation was that she spent over 3 times the predicted time in the shelter. We can further improve our model by getting more dog intake/outcome data, include intake/outcome rates of nearby shelters,  and include other features such as temperment.

Other factors that could contribute to differing adoption rates are the adoption page listed on the Austin Animal Center's website. Certain factors such as search filters, photograph quality, and location on the page could also affect the time spent in the shelter.


Further studies can be done to compare the Austin Animal Center to shelters in other states to compare rates of outcomes. Further studies within the Austin Animal Center can focus on other animals such as cats or wildlife.